In [46]:
# 23.07.10

In [47]:
# HTML/XML => DOM(BeautifulSoup)
#          => CSSOM(selector;pseudo)
# selector(*) { X }
# tags, #id, .class, .class1.class2
# [attr], [attr=value], [attr^=,$=]
# ( ) => 자손, > => 자식, + => 내 다음 형제
# :nth-of-type, :first/last-child, :nth-child, ...
# :has, :not
# => Crawling/Scraping
# while 더이상 url 없을때까지
#   O: robots.txt 파싱(수정)
#   O: HTTP Req/Resp ===> Resp.body (Scraping) => content-type;dom;cssom;re;bytes
#   O: anchor(a;hyper link) => URL(*;urljoin)
#   O: link => regularization(full url)
#   O: visited O/X => URL Pool => ()
#   O: pop(url) => (BFS/DFS/Focused => 탐색전략)

In [48]:
# HyperLinks = ?
# [Tags]
# A, FORM, IMG, EMBEDED, VIDEO, AUDIO,
# META, SCRIPT, STYLE, IFRAME, ...
# [Attributes]
# HREF, SRC, ACTION
# => *[href|src|action]

In [49]:
from requests import request

headers = {
    'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
}

In [50]:
url = 'https://search.naver.com/search.naver'
params = {
    'where':'nexearch',
    'query':'뉴진스'
}

resp = request('GET', url, params=params, headers=headers)

In [53]:
from bs4 import BeautifulSoup

dom = BeautifulSoup(resp.text, 'html5lib')

FeatureNotFound: Couldn't find a tree builder with the features you requested: html5lib. Do you need to install a parser library?

In [ ]:
len(dom.select('[href]')),\
len(dom.select('[src]')),\
len(dom.select('[action]'))
# len(dom.select('[data-]')) ?

In [ ]:
dom.select('[href]')[0].attrs['href'], \
dom.select('[src]')[0].attrs['src'], \
dom.select('[action]')[0].attrs['action'] == '?'

In [ ]:
from requests.compat import urlparse, urljoin

urlparse(resp.url).path,\
urljoin(resp.url, '?a=b'),\
urljoin(resp.url, 'http://www.text.com')

In [ ]:
dom.select('[action]')[0].attrs['method']

In [ ]:
from urllib.parse import parse_qs
parse_qs(urlparse(resp.url).query)

In [ ]:
for node in dom.select('[action]')[0].select('input'):
    print(node.attrs['name']+'='+node.attrs['value'])

In [ ]:
dom.select('[href^]')

In [ ]:
#               root;WWW
#          A            B (10,depth=1)
#      C      D      E(10)       F(400,depth=2)
# => root A B C D E F
#     *   * * * ...
#                   * (leaf)

# BFS => pop(앞에서;Queue) => 
#        관련도순으로...
# DFS => pop(뒤에서;Stack) => 너무 깊으면, 못빠져나옴
# Focused Crawling(전략) => 웹툰, 뉴스, 

In [ ]:
from urllib.robotparser import RobotFileParser
# rb = RobotFileParser(주소)
# rb.read()
# rb.can_fetch(agent, path) T/F

In [ ]:
import re
from requests.exceptions import HTTPError

ROBOT = dict()
        # netloc:[(agent, path, allowT/F),()]

# 1. robots.txt => 중복해서 req X
# 2. ROBOT 1의 url을 key로 물어본거
# 3. 없으면, req/resp
#    3.1 status_code != 200, return True
#    3.2 parsing, User-Agent/allow/disallow
#    3.3 User-agent 물어보고, 지금 작업중인 ua를 ua로 설정
#    3.4 allow/disallow를 key = tuple(ua, path, allow/disallow)
# 4. ROBOT 없으면, 3.1
# 5. 있으면, param.agent값으로, path에 re
#    p = * => [^/]+?, match(p, param.path)
#    5.1 없으면, True
#    5.2 있으면, Sorting(/, /test/, /test/about, /*/about, /test/*.... )

def canFetch(url, agent='*', path='/'):
    k = urlparse(url).netloc
    
    if k not in ROBOT:
        robot = urljoin(url, '/robots.txt')
        resp = request('GET', robot)
    
        if resp.status_code == 200:
            ua = '*'
            ROBOT[k] = list()
            
            for l in resp.text.lower().splitlines():
                a = re.search(r'user-agent:(.+)', l, re.IGNORECASE)

                if a:
                    ua = a.group(1).strip()
                else:
                    p = re.search(
                        r'((?:dis)?allow):(.+)', l, re.IGNORECASE)
                    
                    if p:
                        allow = False if p.group(1) == 'disallow' else True
                        ROBOT[k].append((ua, p.group(2).strip(), allow))
#             rp = RobotFileParser(robot)
#             rp.read()
#             return rp.can_fetch(agent, path)
        
    # robots.txt 파싱이 끝남
    if k in ROBOT:
#         candidates = list()
#         for l in ROBOT[k]:
#             if l[0] == agent and \
#                      re.match(l[1].replace('*', '[^/]+?'), path):
#                 candidates.append(l)
        candidates = list(filter(lambda l:l[0] == agent and \
                     re.match(l[1].replace('*', '[^/]+?'), path),
                                 ROBOT[k]))
        # re.match(* => relace)
        # /*/asdfasd => * => ([^/]+?)
                
        candidates = sorted(candidates, key=lambda c:len(c[1]),
                            reverse=True)
        # *, / = True, /search = False

        if len(candidates) > 0:
            return candidates[0][-1]
        return True
        
    return True

# canFetch('https://lms.sunde41.net', path='/fddd')
canFetch('https://www.google.com', agent='*',
         path='/search/about')
#     try:
#         resp.raise_for_status()
#     except HTTPError as e:
#         d

In [ ]:
# canFetch('https://news.naver.com', agent='*', path='/search')
# canFetch('https://lms.sunde41.net', path='/fddd')
canFetch('https://www.google.com', agent='*', path='/search/?asdfasdf')

In [ ]:
# 원본:[], 전략:[{}], DB
urls = [{'url':'https://www.google.com/search?q=%EB%89%B4%EC%A7%84%EC%8A%A4&oq=%EB%89%B4%EC%A7%84%EC%8A%A4&aqs=chrome.0.0i131i355i433i512j46i131i433i512j0i3j46i131i433i512l2j0i433i512j69i60l2.751j0j4&sourceid=chrome&ie=UTF-8',
         'depth':0}]
visited = []
# 전략:1.depth
focused = 3#[전략](depth, domain<-netloc)

# 종료조건: 더이상 방문할 페이지가 없을때 while urls
while urls:
    # Queue(BFS;FIFO);
    # url pool에서 다음 방문 url 가져오기(1processor, 1thred)
    url = urls.pop(0) # Queue # Stack
    
    # 전략 종료조건:1.depth > threshold
    if url['depth'] > focused:
        visited.append(url)
        continue
        
    # robots.txt 확인(관심있이면 나중에 수정, Naver,Google만 됨)
    try:
        print(urlparse(url['url']).path, \
              canFetch(url['url'], path=urlparse(url['url']).path))
    except:
        print(url['url'])
    
    # 실제 방문 req => resp
    resp = request('GET', url['url'], headers=headers)
    
    # HTML <meta> robots.txt 속성 확인이 있어야 함
    # <meta> index/noindex, follow/nofoolow
#     meta = dom.select_one('meta[name="robots"]')
#     meta.attrs['content'].split(',')

    # 정상적으로 받음
    if resp.status_code == 200:
        # 방문한 목록에 추가
        visited.append(url)
        
        # content-type: text/html or text/xml (지금 관심사)
        #               application/json, images/*
        if re.search('xml|html', resp.headers['content-type']):
            # TEXT(Markup) => DOM
            dom = BeautifulSoup(resp.text, 'lxml')
            
            a = list()
            # Node탐색 CSS Selector(rules;CSSOM)
            # <태그>의 속성(attributes) 중에 href, src가 있는 경우만
            for node in dom.select('[href], [src]'):
                # dom.tag.attrs in href, src
                # temp에는 str 형태의 주소(온전한 X)
                temp = node.attrs['href'
                                  if node.has_attr('href')
                                  else 'src']
                # 링크 필터링(방문할 필요 X)
                # #(위치;내부주소), javascript, data, ...
                if not re.match(r'#|javascript|data:|about:', temp):
                    newurl = urljoin(url['url'], temp)
                    a.append({'url':newurl,'depth':url['depth']+1})
#                 else:
#                     print(temp, urlparse(urljoin(url, temp)))
            print(len(a))
    
            # URL pool 관리
            # 이미 방문한 주소 X, 앞으로 방문할 주소 X
            urls += list(filter(
                lambda link:link['url'] not in [v['url'] for v in visited] \
                and link['url'] not in [u['url'] for u in urls], a))
            # visited, urls => ['url']
            #               => [{'url','depth'}]
            print()
            # 방문이력, url pool 관리
        # else 일때, 처리 X, 500 => 
    # 서버부하, delay
#             a = [urljoin(url, node.attrs['href'
#                         if node.has_attr('href') else 'src'])
#                  for node in dom.select('[href], [src]')]
    # 정상적으로 못받았을때, 400(내 에러),500(쟤 에러)
    else:
        if resp.status_code >= 500:
            # Queue(FIFO)
            urls.append(url)
        else:
            # 방문한 목록
            visited.append(url)
    # 서버 부하 피하기 위해서
#     sleep()

In [ ]:
urlparse('https://search.naver.com/path/?a=b#lnb'), \
urlparse('https://search.naver.com/path?a=b')

In [ ]:
urls[0], visited[-1]

In [ ]:
len(urls), len(visited)

In [ ]:
[urlparse(_).netloc for _ in visited]

In [ ]:
# 23.07.11

In [ ]:
# Crawler(HyperLink 따라다니면서 수집) + DB
# 0. URL pool <= seed(구글 검색결과)
# ===> DFS(Stack, LIFO), BFS(Queue, FIFO)
# ===> Focused(전략:depth,domain제한,id/class제한)
# 1. robots.txt 파싱
#    user-agent\nallow:?\ndisallow:?
#    ---------- tuple => DB(범용관계형X)
# 2. status_code, content-type = text/html or xml
# ===> Scaraping(별도)
# 3. DOM <= bs4, find*, select(css selector)
# 4. HTML Tags[attribute ㅌ href, src, (action-Form, method)]
# 5. Absolute Path(URL/I)
# 6. #, about:, mailto:, data:, javascript, ... => 제거
# 7. 갔다왔는지, 갈예정인지, 둘다 없으면 URL pool 추가
# 8. 0부터시작(URL pool에 더이상 없을때까지)

In [ ]:
from requests import request
from requests.compat import *
from bs4 import BeautifulSoup

headers = {
    'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
}

In [ ]:
import re
from requests.exceptions import HTTPError

ROBOT = dict()

def canFetch(url, agent='*', path='/'):
    k = urlparse(url).netloc
    
    if k not in ROBOT:
        robot = urljoin(url, '/robots.txt')
        resp = request('GET', robot)
    
        if resp.status_code == 200:
            ua = '*'
            ROBOT[k] = list()
            
            for l in resp.text.lower().splitlines():
                a = re.search(r'user-agent:(.+)', l, re.IGNORECASE)

                if a:
                    ua = a.group(1).strip()
                else:
                    p = re.search(
                        r'((?:dis)?allow):(.+)', l, re.IGNORECASE)
                    
                    if p:
                        allow = False if p.group(1) == 'disallow' else True
                        ROBOT[k].append((ua, p.group(2).strip(), allow))
                        
    if k in ROBOT:
        candidates = list(filter(lambda l:l[0] == agent and \
                    re.match(l[1].replace('*', '[^/]+?'), path),
                                ROBOT[k]))
                
        candidates = sorted(candidates, key=lambda c:len(c[1]),
                            reverse=True)

        if len(candidates) > 0:
            return candidates[0][-1]
        return True
        
    return True

In [ ]:
urls = [{'url':'https://search.naver.com/search.naver?where=nexearch&query=%EB%89%B4%EC%A7%84%EC%8A%A4&oquery=%EB%89%B4%EC%A7%84%EC%8A%A4',
        'depth':0}]
visited = []
focused1 = 3
focused2 = ['blog.naver.com']

while urls:
    url = urls.pop(0)
    
    if url['depth'] > focused1:
        visited.append(url)
        continue
        
    try:
        print(urlparse(url['url']).path, \
            canFetch(url['url'], path=urlparse(url['url']).path))
    except:
        print(url['url'])
        
    resp = request('GET', url['url'], headers=headers)
    
    if resp.status_code == 200:
        visited.append(url)
        
        if re.search('xml|html', resp.headers['content-type']):
            dom = BeautifulSoup(resp.text, 'lxml')
            
            a = list()
            for node in dom.select('[href], [src]'):
                temp = node.attrs['href'
                                if node.has_attr('href')
                                else 'src']
                if not re.match(r'#|javascript|data:|about:', temp)\
                and urlparse(temp).netloc in focused2:
                    newurl = urljoin(url['url'], temp)
                    a.append({'url':newurl,'depth':url['depth']+1})
                    
            urls += list(filter(
                lambda link:link['url'] not in [v['url'] for v in visited] \
                and link['url'] not in [u['url'] for u in urls], a))
    else:
        if resp.status_code >= 500:
            urls.append(url)
        else:
            visited.append(url)

In [ ]:
list(map(lambda url:(urlparse(url['url']).path, url['depth']),
        visited))

In [ ]:
import sqlite3

con = sqlite3.connect('blog.db')
cur = con.cursor()

In [ ]:
#             A => TableA
#     1   2    3   4   5 ... => TableB
# TableA
# PK netloc 
# 1  blog.naver.com

# TableB
# path, qs, FK(referer)
# A         1
# A         1
# B         1

# 1 => A (2) => 2/N
# 1 => B (1) => 1/N
# group by path, count() => path별로 들어온 링크의 갯수(inboud)
# sum(FK) => 1번 사이트에서 나온 모든 갯수(Outboud)
# 1 => sum => 전체 링크갯수/N

cur.executescript('''
    DROP TABLE IF EXISTS T_A;
    CREATE TABLE T_A(
        pk INTEGER PRIMARY KEY,
        netloc TEXT NOT NULL UNIQUE
    );
    
    DROP TABLE IF EXISTS T_B;
    CREATE TABLE T_B(
        pk INTEGER PRIMARY KEY,
        path TEXT,
        qs TEXT,
        visited TEXT DEFAULT 'N',
        regdate DATETIME,
        fk INTEGER NOT NULL
    );
''')

In [ ]:
# 기존 테이블 컬럼 추가
cur.execute('''
    ALTER TABLE T_B
    ADD COLUMN visited TEXT DEFAULT 'N'
''')
con.commit()

In [ ]:
url = 'https://search.naver.com/search.naver?where=nexearch&query=%EB%89%B4%EC%A7%84%EC%8A%A4&oquery=%EB%89%B4%EC%A7%84%EC%8A%A4'
cur.execute('''
    INSERT INTO T_A(netloc) VALUES(?)
''', [urlparse(url).netloc])
con.commit()

In [ ]:
cur.execute('''
    INSERT INTO T_B(path, qs, regdate, fk)
    VALUES(?, ?, CURRENT_TIMESTAMP, (
        SELECT PK FROM T_A WHERE netloc=?
    ))
''', [urlparse(url).path, urlparse(url).query, urlparse(url).netloc])
con.commit()

In [ ]:
cur.execute('''
    SELECT T_B.pk, T_A.netloc, T_B.path, T_B.qs FROM T_A
    INNER JOIN T_B
    ON T_B.fk = T_A.pk
    WHERE T_B.visited = 'N'
    ORDER BY T_B.regdate ASC
''')
comp = cur.fetchall()
urlunparse(('https', comp[0][1], comp[0][2], None,
            comp[0][3], None))

In [ ]:
focused = ['blog.naver.com']
limit = 20
while True:
    # 20개 넘어가면 종료
    cur.execute('SELECT COUNT(pk) FROM T_A')
    if cur.fetchone()[0] > 20:
        break
        
    # 더이상 N이 없으면 종료
    cur.execute('''
        SELECT T_B.pk, T_A.netloc, T_B.path, T_B.qs FROM T_A
        INNER JOIN T_B
        ON T_B.fk = T_A.pk
        WHERE T_B.visited = 'N'
        ORDER BY T_B.regdate ASC
        LIMIT 0,1
    ''')
    comp = cur.fetchone()
    
    if not comp:
        break
        
    pk = comp[0]
        
    url = urlunparse(('https', comp[1], comp[2], None, comp[3], None))
    
    try:
        print(urlparse(url).path, \
              canFetch(url, path=urlparse(url).path))
    except:
        print(url)
        
    resp = request('GET', url, headers=headers)
    
    if resp.status_code == 200:
        if re.search('xml|html', resp.headers['content-type']):
            dom = BeautifulSoup(resp.text, 'lxml')
            
            for node in dom.select('[href], [src]'):
                temp = node.attrs['href'
                                  if node.has_attr('href')
                                  else 'src']

                if not re.match(r'#|javascript|data:|about:', temp)\
                   and urlparse(temp).netloc in focused2:
                    nurl = urljoin(url, temp)
                    comp = urlparse(nurl)
                    s = comp.path.split('/')
                    
                    if len(s) > 1: #(netloc/id)
                        cur.execute('''
                            SELECT pk FROM T_A
                            WHERE netloc=?
                        ''', [comp.netloc+'/'+s[1]])
                        
                        if not cur.fetchone():
                            cur.execute('''
                                INSERT INTO T_A(netloc) VALUES(?)
                            ''', [comp.netloc+'/'+s[1]])
                            con.commit()
                            
                        cur.execute('''
                            INSERT INTO T_B(path, qs, regdate, fk)
                            VALUES(?, ?, CURRENT_TIMESTAMP, (
                                SELECT PK FROM T_A WHERE netloc=?
                            ))
                        ''', [comp.path, comp.query, comp.netloc+'/'+s[1]])
                        con.commit()
                        
                        cur.execute('''
                            UPDATE T_B
                            SET visited='Y'
                            WHERE pk=?
                        ''', [pk])
                        con.commit()
                    else:
                        cur.execute('''
                            SELECT pk FROM T_A
                            WHERE netloc=?
                        ''', [comp.netloc])
                        
                        if not cur.fetchone():
                            cur.execute('''
                                INSERT INTO T_A(netloc) VALUES(?)
                            ''', [comp.netloc])
                            con.commit()
                        
                        cur.execute('''
                            INSERT INTO T_B(path, qs, regdate, fk)
                            VALUES(?, ?, CURRENT_TIMESTAMP, (
                                SELECT PK FROM T_A WHERE netloc=?
                            ))
                        ''', [comp.path, comp.query, comp.netloc])
                        con.commit()
                        
                        cur.execute('''
                            UPDATE T_B
                            SET visited='Y'
                            WHERE pk=?
                        ''', [pk])
                        con.commit()
    else:
        if resp.status_code >= 500:
            cur.execute('''
                UPDATE T_B
                SET regdate=CURRENT_TIMESTAMP
                WHERE pk=?
            ''', [pk])
            con.commit()
        else:
            cur.execute('''
                UPDATE T_B
                SET visited='Y'
                WHERE pk=?
            ''', [pk])
            con.commit()

In [ ]:
cur.execute('''
        SELECT * FROM T_A
    ''')
cur.fetchall()

In [ ]:
cur.execute('''
        SELECT * FROM T_B
    ''')
cur.fetchall()

In [ ]:
cur.execute('''
        SELECT T_B.pk, T_A.netloc, T_B.path, T_B.qs FROM T_A
        INNER JOIN T_B
        ON T_B.fk = T_A.pk
        ORDER BY T_B.regdate ASC
        LIMIT 0,1
    ''')
cur.fetchall()

In [ ]:
cur.execute('''
    SELECT fk, COUNT(fk) FROM T_B
    GROUP BY fk
''')
cur.fetchall()

In [ ]:
import numpy as np

In [ ]:
cur.execute('''
    SELECT COUNT(pk) FROM T_A
''')
dim = cur.fetchone()[0]
PR = np.zeros((dim, dim))

In [ ]:
cur.execute('''
    SELECT T_A.netloc, T_A.pk, COUNT(T_B.pk) FROM T_A
    LEFT JOIN T_B
    ON T_B.fk = T_A.pk
    GROUP BY T_B.fk
    ORDER BY T_A.pk
''')
for r in cur.fetchall():
    PR[r[1]-1,:] = r[2]

In [ ]:
SITE = np.zeros((dim,1))
SITE[:,:] = 1/dim

In [ ]:
for _ in range(10):
    SITE = .15 * .85*PR.dot(SITE)

In [ ]:
SITE
# 원래는 위에처럼(PageRank), 이제는 현실적으로 Crawling+Scraping

In [ ]:
urls = ['https://news.daum.net/'] # 앞으로 방문할곳
visited = [] # 이미 방문한곳

while urls: # 더이상 방문할 곳이 없을때까지
    url = urls.pop(0) # 앞에서부터 하나씩 꺼내고(FIFO)
        
    # robots.txt 검사
    try:
        print(urlparse(url).path, \
            canFetch(url, path=urlparse(url).path))
    except:
        print(url)
        
    # HTTP 받아오고
    resp = request('GET', url, headers=headers)
    
    # 잘 받았으면
    if resp.status_code == 200:
        # 방문한곳에 추가해주고
        visited.append(url)
        
        # Scarping(대상: text/html)
        if re.search('html', resp.headers['content-type']):
            # DOM 만들어주고
            dom = BeautifulSoup(resp.text, 'lxml')
            
            # 전략3-1. 특정 영역 - 뉴스 목록
            news = dom.select('''#mainContent + .main-content [href],
                                #mainContent + .main-content [src]''')
            
            # 링크 찾기
            if len(news) > 0:
                a = []
                for temp in news:
                    temp = temp.attrs['href' if temp.has_attr('href') else 'src']
                    if not re.match(r'#|javascript|data:|about:', temp):
                        newurl = urljoin(url, temp)
                        a.append(newurl)

                    urls += list(filter(
                    lambda link:link not in visited and \
                                link not in urls, a))
                    
            # 전략3-2. 뉴스 본문만
            news = dom.select_one('#mArticle')
            
            if news:
                # https://v.daum.net/v/20230711122803039
                with open(url.split('/')[-1].replace('?', '')+'.txt',
                        'w', encoding='utf8') as fp:
                    fp.write(news.text)
        # 전략 3-3. 뉴스 삽화(이미지) 저장
        elif re.search('image', resp.headers['content-type']):
            ext = re.search('(jpeg|png|bmp|webp|gif)',
                            resp.headers['content-type']).group(1)
            with open(url.split('/')[-1].replace('?', '')+'.'+ext, 'wb') as fp:
                    fp.write(resp.content)
    # 못받았으면
    else:
        # FIFO, 앞으로 방문할 곳 제일 뒤에 붙여주고
        if resp.status_code >= 500:
            urls.append(url)
        # 방문했다고 치고, 방문안하게
        else:
            visited.append(url)

In [ ]:
from os import listdir
len(list(filter(lambda f:re.search('[.]txt$', f), listdir('.'))))

In [ ]:
from os import listdir
len(list(filter(lambda f:re.search('[.](!:png|jpeg|bmp|gif|wepb)$', f), listdir('.'))))

In [ ]:
# 23.07.13

In [ ]:
# Crawling+scraping+DB
# url <- url pool(BFS, DFS, Focused)
# 		->DB(netloc, path-page, visited=y/n, redated - 1month)
# while pool empty: ->db.connect(re-connect)
# 	robots.txt 검사
# 		->user-agent; (?:dis)?allow
# 		->/search, /search/about, /search/asdfas, search?param...
# 	[Scraping]
# 	req - resp
# 	resp.headers 확인(status_code, content-type)
# 				=>scraping(text.html, image/*)
# 				=> URL.split('/')[-1], ?, replace(re.sub)
# 	RE, DOM, CSSOM, XPath
# 	HyperLink(a[href], iframe[src], form[method, action])
# 	Link X(#, javascript, data, about, ...)
# 	Normalization(url -> Abosolute Path)
# 	url pool <= path(방문할 곳, 방문한 곳 X)

In [ ]:
from requests import request
from requests.compat import *
from bs4 import BeautifulSoup

headers = {
    'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
}

In [ ]:
url = 'https://pythonscraping.com/pages/javascript/ajaxdemo.html'
resp = requset('GET', url)

In [ ]:
# decording_key = 'bqpNmjWTnx87w7ygUVLKpPVbPtUEhJNorTASFUNbEf7Zl4ofO7ThKV9jJdFcnzEkaQy5P+Uw16BOtszHRvY9OQ=='
# unquote(encording_key) = "bqpNmjWTnx87w7ygUVLKpPVbPtUEhJNorTASFUNbEf7Zl4ofO7ThKV9jJdFcnzEkaQy5P%2BUw16BOtszHRvY9OQ%3D%3D"
# encording key를 unquote로 감싸면 decording_key와 같다

In [ ]:
# 23.07.17

In [ ]:
'''
crawling
scraping - content - type; text/html, application/json/image*...
						-> focused crawling+scraping
dynamaic HTML (AJAX - xmlHTTPRequest/JS/Doc...)
						->DOM(req, resp), data(req/resp -> context-type), 웹툰
cookie/session

http
client		server
req =======> login, sign up  -> RESTful API(put, post) : HTTP body
resp <====== 
            변수=우리정보
cookie		session
		(sessionID)
key-value
'''

In [ ]:
from requests import request
from requests.compat import *
from bs4 import BeautifulSoup

headers = {
    'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
}

In [ ]:
url = 'https://pythonscraping.com/pages/cookies/login.html' 
resp = request('GET', url)

In [ ]:
dom = BeautifulSoup(resp.text, 'html.parser')

In [ ]:
form = dom.select_one('form')

In [ ]:
form.attrs['action']

'welcome.php'

In [ ]:
[tag.attrs['name'] for tag in form.select('[name]')]

['username', 'password']

In [ ]:
urljoin(resp.url, form.attrs['action'])

'https://pythonscraping.com/pages/cookies/welcome.php'

In [ ]:
params = { 
    'username':'testest', 
    'password':'password' 
} 
resp = request(form.attrs['method'], 
               urljoin(resp.url, form.attrs['action']), 
               data = params)

In [ ]:
resp.headers
'''
Client		Server
id/wrong pw -> req
			<- resp (headers cookies X)
id/pw -------> req
		<----- resp (headers set-cookie)
'''

'\nClient\t\tServer\nid/wrong pw -> req\n\t\t\t<- resp (headers cookies X)\nid/pw -------> req\n\t\t<----- resp (headers set-cookie)\n'

In [ ]:
request('GET', urljoin(resp.url, 'profile.php')).text

'You\'re not logged into the site!<br>Visit <a href="login.html">the login page</a> to log in'

In [ ]:
request('GET', urljoin(resp.url, 'profile.php'), 
                    cookies={'loggedin' : '1', 'username' : 'test'}).text

"Hey test! Looks like you're still logged into the site!"

In [ ]:
'''
1. FORM(action, method)
2. FROM tag[name] = tag[value] --> search browser
3. FORM.method, FORM.action, GET; params, POST;data
4. cookie 관리(set-cookie 확인, dict관리)
'''

In [ ]:
resp.request.url, resp.request.body
'''
POST -> body(data)
'''

'\nPOST -> body(data)\n'

In [ ]:
from requests.cookies import cookiejar_from_dict

In [ ]:
c = cookiejar_from_dict({'loggedin' : '1', 'username' : 'whoami'})

In [ ]:
request('GET', urljoin(resp.url, 'profile.php'), 
        cookies=c).text

"Hey whoami! Looks like you're still logged into the site!"

In [ ]:
for row in resp.headers['set-cookie'].split(','): 
    print(row.strip().split('='))

['loggedin', '1']
['username', 'testest']


In [ ]:
from requests.sessions import Session
session = Session()
# '''
# -> set-cookie => cookie jar 관리
# '''

In [ ]:
resp = session.request(form.attrs['method'], 
                       urljoin(resp.url, form.attrs['action']), 
                       data = params) 

In [ ]:
session.request('GET', urljoin(resp.url, 'profile.php')).text

"Hey testest! Looks like you're still logged into the site!"

In [ ]:
url = 'https://lms.sunde41.net' 
resp = request('GET', url) 

In [ ]:
dom = BeautifulSoup(resp.text, 'html.parser') 

In [ ]:
form = dom.select_one('form')

In [ ]:
params = {} 
for tag in form.select('[name]'): 
    params[tag.attrs['name']] = tag.attrs['value'] \
        if tag.has_attr('value') else '' 

In [ ]:
params

{'next': '/', 'email': '', 'password': '', 'remember': ''}

In [ ]:
%%writefile lms.txt 
{ 
    "id":"value", 
    "pw":"pw" 
}

Writing lms.txt


In [ ]:
import json 
# 이건 내꺼 - 작성 X 
with open('lms.txt', 'r', encoding='utf-8') as fp: 
    lms = json.load(fp)

In [ ]:
params['email'] = lms['id'] 
params['password'] = lms['pw']

In [ ]:
session.cookies.clear()

In [ ]:
form.attrs

{'action': '/auth/login',
 'method': 'POST',
 'name': 'login_user_form',
 'class': ['m-form']}

In [ ]:
resp = session.request(form.attrs['method'], 
                    urljoin(resp.url, form.attrs['action']), 
                    data=params)

In [ ]:
dom = BeautifulSoup(session.request('GET', url).text, 'lxml')

FeatureNotFound: Couldn't find a tree builder with the features you requested: lxml. Do you need to install a parser library?

In [ ]:
dom.select('a[href^="/course"]')

[]

In [ ]:
resp = request('GET', urljoin(url, '/course/4'), 
               cookies=session.cookies)

In [ ]:
dom = BeautifulSoup(resp.text, 'lxml')

FeatureNotFound: Couldn't find a tree builder with the features you requested: lxml. Do you need to install a parser library?

In [ ]:
for node in dom.select('a[href^="/static"]'): 
    print(urljoin(resp.url, node.attrs['href'])) 

In [ ]:
resp = request('GET', 'https://lms.sunde41.net/static/uploads/lectures/4/%E1%84%83%E1%85%
root path = / 
           uri 
public path = /uploads/, /static

SyntaxError: unterminated string literal (detected at line 1) (3983802057.py, line 1)

In [ ]:
resp.headers 

{'Server': 'nginx/1.18.0 (Ubuntu)', 'Date': 'Tue, 18 Jul 2023 01:41:37 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Content-Length': '2497', 'Connection': 'keep-alive'}

In [ ]:

c = {} 
for k,v in session.cookies.get_dict().items(): 
    c[k] = v

In [ ]:
request('GET', url, cookies=c).text

'<!DOCTYPE html>\n<html lang="ko">\n<head>\n    <meta charset="UTF-8">\n    <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\n    <meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=1, shrink-to-fit=no">\n    <meta name="description" content="">\n    <meta name="author" content="">\n    <title> 로그인 : 고려대학교 지능정보SW아카데미</title>\n    <link href="/static/vendors/login.css" rel="stylesheet">\n    <link href="/static/vendors/bootstrap.css" rel="stylesheet">\n    <link href="/static/vendors/style.css?v=1.7" rel="stylesheet">\n    <link rel="shortcut icon" href="/static/favicon/favicon.ico" type="image/x-icon">\n    <link rel="icon" href="/static/favicon/favicon.ico" type="image/x-icon">\n    <style>\n        .m-form .m-form__group {\n        padding-top: 0;\n        padding-bottom: 0;\n        }\n    </style>\n</head>\n<body>\n    <div class="form-body without-side">\n        <div class="row">\n            <div class="img-holder">\n                <d

In [ ]:
# auto/login -> /lms.sunde41.net -> /dashboard  
#           302                 30

In [ ]:
resp = request('POST', urljoin(url, '/auth/login'), data=params)

In [ ]:
url = 'https://nid.naver.com/nidlogin.login?mode=form' 
resp = request('GET', url) 

In [ ]:
session.cookies.clear()

In [ ]:
c = {} 
for line in r.splitlines(): 
    if len(line) > 1:
#         print(line.split('\t')[:2]) 
c[line.split('\t')[0]] = line.split('\t')[1]

IndentationError: expected an indented block after 'if' statement on line 3 (4163505514.py, line 5)

In [ ]:
url = 'https://mail.naver.com' 
request('GET', url, cookies=c).text 

'<!DOCTYPE html>\n<html lang="ko">\n<head>\n    <meta charset="UTF-8">\n    <meta http-equiv="X-UA-Compatible" content="IE=edge, chrome=1">\n    <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=no">\n    <meta property="og:type" content="website">\n    <meta property="og:title" content="네이버">\n    <meta property="og:description" content="네이버에 로그인 하고 나를 위한 다양한 서비스를 이용해 보세요">\n    <meta property="og:image" content="https://ssl.pstatic.net/static/nid/account/naver_og_image.png">\n    <meta property="og:image:type" content="image/png">\n    <meta property="og:image:width" content="1200">\n    <meta property="og:image:height" content="630">\n    <title>네이버 : 로그인</title>\n    <link rel="stylesheet" type="text/css" href="/login/css/global/desktop/w_20220216.css?20210812">\n</head>\n<body>\n<div id="wrap" class="wrap">\n    <div class="u_skip"><a href="https://www.naver.com">본문 바로가기</a></div>\n    <header class="header" role="banner" >\n   

In [ ]:
for k,v in c.items(): 
    session.cookies.set(k,v)

In [ ]:
import re 
re.search(r'<title>(.+)</title>', session.request('GET', url).text)

<re.Match object; span=(689, 713), match='<title>네이버 : 로그인</title>'>

In [ ]:
# 받은메일함
session.get('https://mail.naver.com/v2/folders/0').text

'<!DOCTYPE html>\n<html lang="ko">\n<head>\n    <meta charset="UTF-8">\n    <meta http-equiv="X-UA-Compatible" content="IE=edge, chrome=1">\n    <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=no">\n    <meta property="og:type" content="website">\n    <meta property="og:title" content="네이버">\n    <meta property="og:description" content="네이버에 로그인 하고 나를 위한 다양한 서비스를 이용해 보세요">\n    <meta property="og:image" content="https://ssl.pstatic.net/static/nid/account/naver_og_image.png">\n    <meta property="og:image:type" content="image/png">\n    <meta property="og:image:width" content="1200">\n    <meta property="og:image:height" content="630">\n    <title>네이버 : 로그인</title>\n    <link rel="stylesheet" type="text/css" href="/login/css/global/desktop/w_20220216.css?20210812">\n</head>\n<body>\n<div id="wrap" class="wrap">\n    <div class="u_skip"><a href="https://www.naver.com">본문 바로가기</a></div>\n    <header class="header" role="banner" >\n   

In [ ]:
url = 'https://mail.naver.com/json/list' 
params = { 
    'folderSN':'0', 
    'page':'1', 
    'viewMode':'time', 
    'previewMode':'1', 
    'sortField':'1', 
    'sortType':'0', 
    'u':'gon0121' 
} # params['u'] <= 본인 아이디 
resp = session.request('POST', url, params=params)

In [ ]:
len(resp.json()['mailData'])

KeyError: 'mailData'

In [ ]:
resp = request('POST', url, params=params, cookies=c) 
len(resp.json()['mailData']) 
# 스팸 메일함 불러오기

KeyError: 'mailData'

In [ ]:
resp = session.request('GET', 'https://cafe.naver.com')

In [ ]:
resp.text 
# 카페 목록 불러오기

'{"Result":"FAIL","Message":"", "LoginStatus":"NOLOGIN"}'

In [ ]:
url = 'https://www.ypbook.co.kr/kor_index.yp'
resp = request('GET', url)
dom = BeautifulSoup(resp.text, 'lxml')

ConnectTimeout: HTTPSConnectionPool(host='www.ypbook.co.kr', port=443): Max retries exceeded with url: /kor_index.yp (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7f091dcc30a0>, 'Connection to www.ypbook.co.kr timed out. (connect timeout=None)'))

In [ ]:
form = dom.select('form')[-1]
params = {node.attrs['name'];node.attrs['value']
        for node in form.select(['[name]'])}
params['query'] = '뉴진스'

In [ ]:
form.attrs

In [ ]:
params = { 
    'query': '뉴진스', 
    'collection': 'ALL', 
    'searchfield': 'ALL', 
    'sortField': 'DATE', 
    'notSoldOut': 'N', 
    'catesearch': 'false', 
    'viewStyle': 'list', 
    'pageNum': '1' 
} 
resp = request('POST', 'https://www.ypbooks.co.kr/ypbooks/search/requestAjaxSearchTab.jsp'
            data=params)

In [ ]:
'''
HTTP(Req/Resp) 
RE,DOM,CSSOM 
DHTML/AJAX(XHR) 
Session/Cookie(*) 
=> Crawling + Scraping 
뉴스, 이미지, 메일, 소셜미디어, ...
'''

''

In [ ]:
from datetime import datetime

In [ ]:
datetime.now().timestamp()

1689644679.912169

In [ ]:
datetime.fromtimestamp(1689644679)

datetime.datetime(2023, 7, 18, 10, 44, 39)

In [ ]:
t = int(datetime.now().timestamp()) 
pw = 'toinstagram' 
f'#PWD_INSTAGRAM_BROWSER:0:{t}:{pw}' 

'#PWD_INSTAGRAM_BROWSER:0:1689644723:toinstagram'

In [ ]:
url = 'https://www.instagram.com/api/v1/web/accounts/login/ajax/' 
params = { 
    'enc_password': '#PWD_INSTAGRAM_BROWSER:10:1689566089:AVxQAGTw2LChRHhfhxoed0/Y8FxG6+FciohZQIujcJ3LoEMW+cqyeHMVdp/tvjtlZL2pbt3yrVHOIJOlwWJOjtfvZ8LipOtQuSWiasSifvscqoly08WfAU+SuIlIEtjXn9NZKSkbWGJaz1HyZUpWNHAB'
    'optIntoOneTap': 'false', 
    'queryParams': '{}', 
    'trustedDeviceRecords': '{}', 
    'username': 'test@test.com' 
} 
resp = request('POST', url, data=params) 
resp.text

SyntaxError: invalid syntax (3922543809.py, line 4)

In [ ]:
'''
Oauth, CSRF 
API - 내가(인증) 
API(인증)
'''

In [ ]:
url = 'https://www.instagram.com' 
resp = request('GET', url)

In [ ]:
csrf = re.search(r'\\"csrf_token\\":\\"([^\\]+)', resp.text).group(1) 

NameError: name 're' is not defined

In [ ]:
url = 'https://www.instagram.com/api/v1/web/accounts/login/ajax/' 
params = { 
    'enc_password': '#PWD_INSTAGRAM_BROWSER:10:1689566089:AVxQAGTw2LChRHhfhxoed0/Y8FxG6+FciohZQIujcJ3LoEMW+cqyeHMVdp/tvjtlZL2pbt3yrVHOIJOlwWJOjtfvZ8LipOtQuSWiasSifvscqoly08WfAU+SuIlIEtjXn9NZKSkbWGJaz1HyZUpWNHAB'
    'optIntoOneTap': 'false', 
    'queryParams': '{}', 
    'trustedDeviceRecords': '{}', 
    'username': '본인아이디' 
} 
t = int(datetime.now().timestamp()) 
pw = '비밀번호' 
params['enc_password'] = f'#PWD_INSTAGRAM_BROWSER:0:{t}:{pw}' 
resp = request('POST', url, data=params, 
            headers={'x-csrftoken':csrf}) 
resp.text

SyntaxError: invalid syntax (2617907614.py, line 4)

In [ ]:
# 23.07.18

In [ ]:
'''
Session(Cookie) -> 인증된 사용자 정보 -> 서버 안쪽 데이터 
- [Name:Value];domain/path/expired... 
- Cookie -> Client Browser(temporal path;webdb...) 
-> requests.request*(get,post,...) **cookies = {k:v} 
	cookie jar object <= cookie_jar_from_dict 
	cookie 매번 관리(set-cookie) 
-> Session().request*(get,post,...) 
	request vs. session.request ; cookie 관리 주체 
-> Brwoser cookie => cookie dict, jar 
e.g., LMS(RESTful API), Naver(Mail, Cafe, ..) 
		SNS(Instagram) ; API(인증X,허가X,) 
		cookie;header;X-CSRF-TOKEN <- /(RE) 
		params['enc_pw'] => #PWD_INSTAGRAM:?(보안레벨):
'''

''

In [ ]:
from requests import request 
from requests.compat import * 
from bs4 import BeautifulSoup 
import re 
headers = { 
    'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
}

In [ ]:
from requests.sessions import Session

In [ ]:
url = 'https://www.instagram.com' 
session = Session() 
resp = session.request('GET', url)

In [ ]:
CSRF = re.search(r'csrf_token\\":\\"(.+?)\\"', 
                resp.text).group(1)

In [ ]:
ajax = 'https://www.instagram.com/api/v1/web/accounts/login/ajax/' 
params = { 
    'enc_password': '', 
    'optIntoOneTap': 'false', 
    'queryParams': '{}', 
    'trustedDeviceRecords': '{}', 
    'username': '' 
} 
params['username'] = 'gon0121+dev@gmail.com'	# 내 아이디

In [ ]:
from datetime import datetime 
T = int(datetime.now().timestamp()) 
PW = 'toinstagram' # 본인 비밀번호 
#PWD_INSTAGRAM_BROWSER:보안레벨(숫자):시간(숫자):PW 
f'#PWD_INSTAGRAM_BROWSER:0:{T}:{PW}'

'#PWD_INSTAGRAM_BROWSER:0:1689820631:toinstagram'

In [ ]:
params['enc_password'] = f'#PWD_INSTAGRAM_BROWSER:0:{T}:{PW}' 
params

{'enc_password': '#PWD_INSTAGRAM_BROWSER:0:1689820631:toinstagram',
 'optIntoOneTap': 'false',
 'queryParams': '{}',
 'trustedDeviceRecords': '{}',
 'username': 'gon0121+dev@gmail.com'}

In [ ]:
resp = session.post(ajax, data=params)

In [ ]:
resp.text

'{"message":"CSRF token missing or incorrect","status":"fail"}'

In [ ]:
len(session.cookies.get('csrftoken'))

32

In [ ]:
headers = { 
    'x-csrftoken':CSRF 
} 
resp = request('POST', ajax, data=params, 
               headers=headers)

In [ ]:
resp.text

'{"user":true,"userId":"49814451599","authenticated":true,"oneTapPrompt":true,"status":"ok"}'

In [ ]:
headers['x-csrftoken'] = session.cookies.get('csrftoken') 
resp = session.post(ajax, data=params, 
                    headers=headers) 
resp.text

'{"user":true,"userId":"49814451599","authenticated":true,"oneTapPrompt":true,"status":"ok"}'

In [ ]:
session.cookies.get('csrftoken')

'shgUjEiqIb85KdsuwnNrPhK8dCYghBZQ'

In [ ]:
CSRF

'xpRWWrhkoi0EeM1VwXoltJTwU8VNxere'

In [ ]:
resp.headers['set-cookie'] 

'csrftoken=shgUjEiqIb85KdsuwnNrPhK8dCYghBZQ; Domain=.instagram.com; expires=Thu, 18-Jul-2024 02:37:20 GMT; Max-Age=31449600; Path=/; Secure, rur="PRN\\05449814451599\\0541721356640:01f78a8b89950810e7b5aa27701c33de15a90f8396028ab8c4ee5e7164e3d0330fc7f2a1"; Domain=.instagram.com; HttpOnly; Path=/; SameSite=Lax; Secure, ds_user_id=49814451599; Domain=.instagram.com; expires=Wed, 18-Oct-2023 02:37:20 GMT; Max-Age=7776000; Path=/; Secure, sessionid=49814451599%3AnNwdNJoozFQSXo%3A28%3AAYfkrF64xwG84-rT3ffEukTJ-JYHBZsS19rrDrt-qw; Domain=.instagram.com; expires=Fri, 19-Jul-2024 02:37:20 GMT; HttpOnly; Max-Age=31536000; Path=/; Secure'

In [ ]:
search = 'https://www.instagram.com/api/v1/web/search/topsearch/' 
params = { 
    'context':'blended', 
    'include_reel':'true', 
    'query':'', 
    'rank_token':'0.5946241399410475', 
	'search_surface':'web_top_search' 
}

In [ ]:
params['query'] = '뉴' 
headers['x-csrftoken'] = session.cookies.get('csrftoken') 
resp = session.get(search, params=params, 
                    headers=headers) 
resp.text 

'{"users":[{"position":0,"user":{"has_anonymous_profile_picture":false,"fbid_v2":17841442336192889,"pk":"42343115206","pk_id":"42343115206","username":"official.newjeans","full_name":"NewJeans 뉴진스","is_private":false,"is_verified":false,"profile_pic_id":"2887334722373425484_42343115206","profile_pic_url":"https://scontent-gmp1-1.cdninstagram.com/v/t51.2885-19/294503123_1663687497334645_7728765606903984238_n.jpg?stp=dst-jpg_s150x150\\u0026_nc_ht=scontent-gmp1-1.cdninstagram.com\\u0026_nc_cat=101\\u0026_nc_ohc=Z6u93GM0C7cAX8uEj6r\\u0026edm=AHG7ALcBAAAA\\u0026ccb=7-5\\u0026oh=00_AfB2fTfBGr0V6PUT_3KtyEBQ3L2T3wSzVuRKF34tT4RNfg\\u0026oe=64BD0F57\\u0026_nc_sid=c9086e","has_opt_eligible_shop":false,"account_badges":[],"birthday_today_visibility_for_viewer":"NOT_VISIBLE","third_party_downloads_enabled":0,"friendship_status":{"following":false,"is_private":false,"incoming_request":false,"outgoing_request":false,"is_bestie":false,"is_restricted":false,"is_feed_favorite":false},"latest_reel_media"

In [ ]:
params['query'] = '뉴진스' 
headers['x-csrftoken'] = session.cookies.get('csrftoken') 
resp = session.get(search, params=params, 
                    headers=headers) 
resp.text

'{"users":[{"position":1,"user":{"has_anonymous_profile_picture":false,"fbid_v2":17841442336192889,"pk":"42343115206","pk_id":"42343115206","username":"official.newjeans","full_name":"NewJeans 뉴진스","is_private":false,"is_verified":false,"profile_pic_id":"2887334722373425484_42343115206","profile_pic_url":"https://scontent-gmp1-1.cdninstagram.com/v/t51.2885-19/294503123_1663687497334645_7728765606903984238_n.jpg?stp=dst-jpg_s150x150\\u0026_nc_ht=scontent-gmp1-1.cdninstagram.com\\u0026_nc_cat=101\\u0026_nc_ohc=Z6u93GM0C7cAX8uEj6r\\u0026edm=AHG7ALcBAAAA\\u0026ccb=7-5\\u0026oh=00_AfB2fTfBGr0V6PUT_3KtyEBQ3L2T3wSzVuRKF34tT4RNfg\\u0026oe=64BD0F57\\u0026_nc_sid=c9086e","has_opt_eligible_shop":false,"account_badges":[],"birthday_today_visibility_for_viewer":"NOT_VISIBLE","third_party_downloads_enabled":0,"friendship_status":{"following":false,"is_private":false,"incoming_request":false,"outgoing_request":false,"is_bestie":false,"is_restricted":false,"is_feed_favorite":false},"latest_reel_media"

In [ ]:
resp.json()['users'][0]['user']['pk']

'42343115206'

In [ ]:
user = 42343115206#resp.json()['users'][0]['user']['pk'] 
feed = f'https://www.instagram.com/api/v1/feed/user/{user}/?' 
params = { 
    'count':'12' 
} 
headers['user-agent'] = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
headers['x-csrftoken'] = session.cookies.get('csrftoken') 
resp = session.get(feed, params=params, 
                    headers=headers) 
resp.text 

'{"message":"useragent mismatch","status":"fail"}'

In [ ]:
# 1. UA 
# 2. 검색: 키워드(PK) 
# 3. User, Tag 
# 4. If User, 게시물 첫 페이지 
# 5. 다음 페이지

In [ ]:
# selenium 사용하기

In [ ]:
from selenium import webdriver
# driver = webdriver.Chrome('driver의경로')


In [ ]:
driver = webdriver.Chrome()

The version of chrome cannot be detected. Trying with latest driver version


WebDriverException: Message: Service /home/roina/.cache/selenium/chromedriver/linux64/114.0.5735.90/chromedriver unexpectedly exited. Status code was: 127


In [ ]:
driver.current_window_handle

In [ ]:
# !pip install webdriver‐manage

In [ ]:
from webdriver_manager.chrome import ChromeDriverManager 
driver = webdriver.Chrome(ChromeDriverManager().install()) 

In [ ]:
driver.get('https://lms.sunde41.net') 

In [ ]:
from selenium.webdriver.common.by import By

In [ ]:
# By.CSS_SELECTOR, By.XPATH

In [ ]:
for _ in driver.find_elements(By.CSS_SELECTOR, 'form input[name]'): 
    print(_.tag_name, _.text, _.get_attribute('class'))

NameError: name 'driver' is not defined

In [ ]:
# dom1 = resp.text 
dom2 = BeautifulSoup(driver.page_source, 'html.parser')
# dom1(resp.raw => dom) != dom2(renderin

In [ ]:
dom2.select('form input[name]')[0], 
driver.find_elements(By.CSS_SELECTOR, 'form input[name]')[0] 

In [ ]:
type(dom2.select('form input[name]')[0]), \ 
type(driver.find_elements(By.CSS_SELECTOR, 'form input[name]')[0]) 

In [ ]:
tags = driver.find_elements(By.CSS_SELECTOR, 
                            'form #email, form #password')

In [ ]:
tags[0].clear() 
tags[0].send_keys('gon0121@gmail.com')

In [ ]:
tags[0].get_attribute('value')

In [ ]:
import json 
with open('lms.txt', 'r') as f: 
    lms = json.load(f)

In [ ]:
tags[1].send_keys(lms['pw']) 

In [ ]:
driver.find_element(By.CSS_SELECTOR, '#button').click()

NameError: name 'driver' is not defined

In [ ]:
session.cookies.clear()

In [ ]:
for c in driver.get_cookies(): 
    session.cookies.set(c['name'], c['value'])

In [ ]:
session.get('https://lms.sunde41.net').text

In [ ]:
driver.delete_all_cookies()

In [ ]:
driver.refresh()

In [ ]:
email = driver.find_element(By.XPATH, '//form//input[@id="email"]') 
# /form, //form 

In [ ]:
pw = driver.find_element(By.XPATH, '//form//input[@id="password"]'

In [ ]:
from selenium.webdriver.common.keys import Keys

In [ ]:
email.clear() 
email.send_keys(lms['id']) 
pw.send_keys(lms['pw'])

NameError: name 'email' is not defined

In [ ]:
driver.find_element(By.XPATH, '//input[@id="button"]').click()
# email.is_displayed() => DOM 변경(ID값 참조하지 못함)

In [ ]:
driver.find_element(By.XPATH, '//ul//ul/li').click()

In [ ]:
for a in driver.find_elements(By.CSS_SELECTOR, 'a[href$=".pdf"]'): 
    print(a.tag_name, a.text, a.is_displayed())

In [ ]:
driver.find_elements(By.CSS_SELECTOR, 'a[href$=".pdf"]')[0].click()

In [ ]:
driver.find_elements(By.CSS_SELECTOR, 
                     'a:has( + div a[href$=".pdf"])')[0].click()

In [ ]:
driver.find_elements(By.CSS_SELECTOR, 'a[href$=".pdf"]')[0].click()

In [ ]:
# s_enabled,      is_selected,     is_displayed 
# :button 활성화여부  : 선택박스 선택여부  :코드상에 화면에있는지 여부

In [ ]:
driver.execute_script('alert("test");')

In [ ]:
driver.execute_script('toggleFullScreen();')	# => 무조건 사용

In [ ]:
driver.execute_async_script() 

In [ ]:
driver.get('https://www.naver.com') 

In [ ]:
driver.find_element(By.XPATH, '//div[@id="account"]//a').click()

In [ ]:
driver.find_element(By.XPATH, '//input[@id="id"]')

In [ ]:
driver.find_element(By.XPATH, '//input[@id="pw"]')

In [ ]:
driver.find_element(By.XPATH, '//form//button[@type="submit"]')

In [ ]:
# CAPTCHA => reCAPTCHA; Collective Intelligence 
# => bot 방지

In [ ]:
cookie = {c['name']:c['value'] for c in driver.get_cookies()}

In [ ]:
request('GET', 'https://mail.naver.com', cookies=cookie).text 

In [ ]:
driver.add_cookie() 

In [ ]:
driver.get('https://mail.naver.com')

In [ ]:
logout = driver.find_element(By.XPATH, '//a[@id="gnb_logout_button"]')

In [ ]:
logout.is_displayed()

In [ ]:
logout.click()	#ERROR

NameError: name 'logout' is not defined

In [ ]:
from time import sleep # => Sleep => process 멈춰 
driver.implicitly_wait # => IW =>  
from selenium.webdriver.support.wait import WebDriverWait# => EW 
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
sleep(10)

NameError: name 'sleep' is not defined

In [ ]:
driver.implicitly_wait(10) 

NameError: name 'driver' is not defined

In [ ]:
logout.click()

NameError: name 'logout' is not defined

In [ ]:
wait = WebDriverWait(driver, 10, .5)

NameError: name 'WebDriverWait' is not defined

In [ ]:
wait.until(EC.element_to_be_clickable(logout)) 

NameError: name 'wait' is not defined

In [ ]:
# DOM 1개 여러개(DOM‐DOM, TABs DOMs) 

In [ ]:
# driver.switch_to.frame() 
driver.switch_to.window(driver.window_handles[2])

NameError: name 'driver' is not defined

In [ ]:
driver.window_handles

NameError: name 'driver' is not defined

In [ ]:
driver.current_window_handle 

NameError: name 'driver' is not defined

In [ ]:
driver.current_url 

NameError: name 'driver' is not defined

In [ ]:
iframe = driver.find_element(By.XPATH, '//iframe') 

NameError: name 'driver' is not defined

In [ ]:
driver.switch_to.frame(iframe)

NameError: name 'driver' is not defined

In [ ]:
driver.switch_to.default_content()

NameError: name 'driver' is not defined

In [ ]:
driver.page_source

NameError: name 'driver' is not defined